In [2]:
#pip install seaborn
import warnings
warnings.filterwarnings('ignore')
from evaluation.generalevaluator import *
from modelsdefinition.GATE import GATE
from factory import create_data_loader
import pandas as pd

from pytorch_tabular import TabularModel

In [2]:
data_loader = create_data_loader('ageconditions', test_size=0.2, normalize_features = "mean_std", return_extra_info = True)
X_train, X_val, y_train, y_val, extra_info = data_loader.load_data()

In [6]:
model = TabularModel.load_model(dir = "../output/modelsaves/ageconditions/GATE/202307-1622-2744-6b41faa3-9de4-4bf3-9e73-62cac06161ec", strict = False)

2023-07-16 22:29:46,508 - {pytorch_tabular.tabular_model:129} - INFO - Experiment Tracking is turned off
2023-07-16 22:29:46,512 - {pytorch_tabular.tabular_model:268} - INFO - Preparing the Trainer
/Users/mdicio/.pyenv/versions/3.10.0/envs/WTab/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:589: LightningDeprecationWarning: The Trainer argument `auto_select_gpus` has been deprecated in v1.9.0 and will be removed in v2.0.0. Please use the function `pytorch_lightning.accelerators.find_usable_cuda_devices` instead.
  rank_zero_deprecation(
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [18]:
df = pd.read_csv(
    "./data/kaggle/icr-identify-age-related-conditions/train.csv"
).drop(columns=["Id"])

df_test = pd.read_csv("./data/kaggle/icr-identify-age-related-conditions/test.csv")


# map the values to 0 and 1
df["EJ"] = df["EJ"].map({"A": 1, "B": 0})
df_test["EJ"] = df_test["EJ"].map({"A": 1, "B": 0})

# find columns with NaN values
cols_with_nans = df.columns[df.isna().any()].tolist()

# fill NaN values in those columns with median
for col in cols_with_nans:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)
  
# find columns with NaN values
cols_with_nans = df_test.columns[df_test.isna().any()].tolist()

# fill NaN values in those columns with median from df
for col in cols_with_nans:
    median_val = df[col].median()
    df_test[col] = df_test[col].fillna(median_val)

In [24]:
df_test

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
predictions = model.predict(df_test)

Output()

In [28]:
predictions

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FS,GB,GE,GF,GH,GI,GL,0_probability,1_probability,prediction
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.415467,0.584533,1
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.415467,0.584533,1
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.415467,0.584533,1
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.415467,0.584533,1
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.415467,0.584533,1


In [25]:
submission = predictions[["Id", "0_probability", "1_probability"]].rename(columns={'0_probability': 'class_0', '1_probability': 'class_1'})

In [27]:
submission

,Id,class_0,class_1
0,00eed32682bb,0.415467,0.584533
1,010ebe33f668,0.415467,0.584533
2,02fa521e1838,0.415467,0.584533
3,040e15f562a2,0.415467,0.584533
4,046e85c7cc7f,0.415467,0.584533


In [15]:
len(df_test)

5

In [12]:
df_test

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL
0,00eed32682bb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010ebe33f668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,02fa521e1838,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,040e15f562a2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,046e85c7cc7f,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
y_true = validation["target"]
y_pred = tabular_model.predict(validation)["prediction"]
evaluator = Evaluator(
                y_true=y_true,
                y_pred=y_pred,
                run_metrics=["mse", "f1", "accuracy"],
                metric="mse",
                problem_type="multiclass_classification",
            )
output_metrics = evaluator.evaluate_model()



Output()

# GATE Tests

In [ ]:
help(GatedAdditiveTreeEnsembleConfig)

In [ ]:
help(OptimizerConfig)

In [3]:
help(TrainerConfig)

NameError: name 'TrainerConfig' is not defined

In [5]:
import torch.optim as optim
data_config = DataConfig(
    target=['target'],
    continuous_cols= [i for i in extra_info["num_col_names"] if i != "target"],
    categorical_cols=extra_info["cat_col_names"],
    #num_workers = 4
)
trainer_config = TrainerConfig(
    auto_lr_find=False, # Runs the LRFinder to automatically derive a learning rate
    batch_size=16,
    max_epochs=300,
    early_stopping="valid_loss", # Monitor valid_loss for early stopping
    early_stopping_mode = "min", # Set the mode as min because for val_loss, lower is better
    early_stopping_patience=20, # No. of epochs of degradation training will wait before terminating
    checkpoints="valid_loss", # Save best checkpoint monitoring val_loss
    load_best=True, # After training, load the best checkpoint
)

optimizer_config = OptimizerConfig(
    optimizer="Adam",
    optimizer_params={
        "weight_decay": 0.001
    },
    lr_scheduler="ReduceLROnPlateau",
    lr_scheduler_params={
        "mode": "min",
        "factor": 0.1,
        "patience": 5,
        "verbose": True
    },
    lr_scheduler_monitor_metric="valid_loss"
)

model_config = GatedAdditiveTreeEnsembleConfig(
    task="classification",
    tree_depth  =  5,
    num_trees   =  12,
    chain_trees = False, # akin to bagging, True is akin to boosting
    gflu_stages =  2
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

2023-07-16 19:04:07,383 - {pytorch_tabular.tabular_model:102} - INFO - Experiment Tracking is turned off


In [16]:
from sklearn.utils.class_weight import compute_class_weight

In [21]:
np.unique(y_train.values)

array([0, 1, 2])

In [22]:
help(compute_class_weight)

Help on function compute_class_weight in module sklearn.utils.class_weight:

compute_class_weight(class_weight, *, classes, y)
    Estimate class weights for unbalanced datasets.
    
    Parameters
    ----------
    class_weight : dict, 'balanced' or None
        If 'balanced', class weights will be given by
        ``n_samples / (n_classes * np.bincount(y))``.
        If a dictionary is given, keys are classes and values
        are corresponding class weights.
        If None is given, the class weights will be uniform.
    
    classes : ndarray
        Array of the classes occurring in the data, as given by
        ``np.unique(y_org)`` with ``y_org`` the original class labels.
    
    y : array-like of shape (n_samples,)
        Array of original class labels per sample.
    
    Returns
    -------
    class_weight_vect : ndarray of shape (n_classes,)
        Array with class_weight_vect[i] the weight for i-th class.
    
    References
    ----------
    The "balanced" heurist

In [20]:

class_weights= compute_class_weight('balanced',np.unique(y_train.values),y_train.values)
class_weights=torch.tensor(class_weights,dtype=torch.float)
 
print(class_weights) #([1.0000, 1.0000, 4.0000, 1.0000, 0.5714])
#Then pass it to nn.CrossEntropyLoss's weight variable

weighted_loss = nn.CrossEntropyLoss(weight=class_weights,reduction='mean')

TypeError: compute_class_weight() takes 1 positional argument but 3 were given

In [12]:
# Calculate the weighted loss
weighted_loss = get_class_weighted_cross_entropy(y_train.values.ravel(), mu =1.4)



In [13]:
assert X_train.isnull().sum().sum() == 0
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

# Merge X_train and y_train  
train = pd.concat([X_train, y_train], axis=1)

# Merge X_val and y_val
validation = pd.concat([X_val, y_val], axis=1)

tabular_model.fit(
    train=train, 
    validation=validation,
    loss=weighted_loss
)

2023-07-13 18:05:17,886 - {pytorch_tabular.tabular_model:102} - INFO - Experiment Tracking is turned off
Global seed set to 42
2023-07-13 18:05:17,913 - {pytorch_tabular.tabular_model:465} - INFO - Preparing the DataLoaders
2023-07-13 18:05:17,916 - {pytorch_tabular.tabular_datamodule:286} - INFO - Setting up the datamodule for classification task
2023-07-13 18:05:17,941 - {pytorch_tabular.tabular_model:508} - INFO - Preparing the Model: GatedAdditiveTreeEnsembleModel
2023-07-13 18:05:18,073 - {pytorch_tabular.tabular_model:264} - INFO - Preparing the Trainer
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2023-07-13 18:05:18,142 - {pytorch_tabular.tabular_model:566} - INFO - Training Started


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                       ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ custom_loss      │ CrossEntropyLoss           │      0 │
│ 1 │ _backbone        │ GatedAdditiveTreesBackbone │ 28.1 K │
│ 2 │ _embedding_layer │ Embedding1dLayer           │      8 │
│ 3 │ _head            │ CustomHead                 │    111 │
└───┴──────────────────┴────────────────────────────┴────────┘

Trainable params: 28.2 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 28.2 K                                                                                               
Total estimated model params size (MB): 0

Output()

Epoch 00138: reducing learning rate of group 0 to 1.0000e-04.

Epoch 00149: reducing learning rate of group 0 to 1.0000e-05.

Epoch 00156: reducing learning rate of group 0 to 1.0000e-06.

Epoch 00168: reducing learning rate of group 0 to 1.0000e-07.

2023-07-13 18:13:07,327 - {pytorch_tabular.tabular_model:568} - INFO - Training the model completed
2023-07-13 18:13:07,328 - {pytorch_tabular.tabular_model:1207} - INFO - Loading the best model


In [14]:
y_true = validation["target"]
y_pred = tabular_model.predict(validation)["prediction"]
evaluator = Evaluator(
                y_true=y_true,
                y_pred=y_pred,
                run_metrics=["mse", "f1", "accuracy"],
                metric="mse",
                problem_type="multiclass_classification",
            )
output_metrics = evaluator.evaluate_model()

Output()

In [15]:
output_metrics

{'mse': 0.0, 'accuracy': 1.0, 'f1': 1.0}